# Building a GraphLab Matrix Factorization Recommender

Yeah, fine, the starter-code for a popularity recommender wasn't that exciting. Let's see if we can tune a GraphLab recommender to do better!

**Spoiler Alert**: Yes, we can.

In [1]:
import numpy as np
import pandas as pd

import graphlab as gl

import scoring

### Load the dataset

In [2]:
training_set = pd.read_csv('jester_train.csv')
test_set = pd.read_csv('jester_test.csv')

In [3]:
# Sanity check:
training_set.head()

,user_id,joke_id,rating
0,7302,29,7.156
1,61815,46,6.375
2,31128,96,2.281
3,36125,147,-1.781
4,18007,60,2.188


In [4]:
# Sanity check:
test_set.head()

,user_id,joke_id
0,30762,24
1,54667,128
2,38515,68
3,44643,39
4,58677,13


### A word or two about overfitting

Matrix Factorization models can easily overfit the data due to the HUGE number of parameters which compose the model. In fact, it's likely you'll have _more_ parameters in your factorization model than you have ratings in your dataset! The little overfitting buzzer in your brain should be louder than ever.

Let's think about how we can remove variance (recall the bias-variance tradeoff) from our model in order to not overfit (as much, at least). How can we lower the variance of our factorization model? (pause and think about it)

There are two obvious ways:
1. Decrease `k` (the number of latent factors), and/or
2. Increase `lambda` (the regularization tuning parameter; note that GraphLab exposes two regularization terms to us).

Another thought: If we wanted to increase the number of latent factors (i.e. increase `k`) we can do that, but we'll need EVEN MORE regularization to offset the variance introduced by having more latent factors.

So, here we go. Below is how to train and use a GraphLab matrix factorization recommender. We'll start with (mostly) the defaults, but I've indicated below which hyperparameters should be tuned (which we'll do later in this notebook).

See documentation [here](https://turi.com/products/create/docs/generated/graphlab.recommender.factorization_recommender.create.html).

In [5]:
# Train on the training set.
factor_rec = gl.factorization_recommender.create(observation_data=gl.SFrame(training_set),
                                                 user_id="user_id",
                                                 item_id="joke_id",
                                                 target='rating',
                                                 num_factors=8,                  # <-- we'll tune this later!
                                                 regularization=1e-08,           # <-- we'll tune this later!
                                                 linear_regularization=1e-10,    # <-- we'll tune this later!
                                                 max_iterations=100,
                                                 solver='auto',
                                                 random_seed=12345,
                                                 verbose=True)

# Predict on the test set.
factor_predictions = test_set.copy()
factor_predictions['rating'] = factor_rec.predict(gl.SFrame(test_set))

# Output the test set RMSE.
# This will help us know how much we're overfitting by comparing it to the RMSE of the _training_ set.
print 'Test set RMSE:', scoring.score_rmse(factor_predictions)

# Finally, output our model's _real_ score according to the top-5-percent scoring metric used by this repo.
print 'Factorization recommender score:', scoring.score_top_5_percent(factor_predictions)

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1485392592.log


Recsys training: model = factorization_recommender

Preparing data set.

Data has 1000000 observations with 58125 users and 140 items.

Data prepared in: 0.574595s

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 7.95361

Final training RMSE: 2.81959

Test set RMSE: 5.03042742439
Factorization recommender score: 2.21201084437


The model above isn't doing very well. Looking at the RMSE of the training set (2.81959) compared to the RMSE of the test set (5.0304), it looks like it is overfitting the training data way too much. We can deal with that by (1) decreasing `k`, and/or (2) increasing the regularization tuning parameters.

The other major indication that we're doing something wrong is that our top-5-percent metric is WORSE than the Popularity Recommender! Yikes, all this work, and we're doing worse than before...

Next, let's search for some hyperparameters which give better results. Once we find those hyperparameters, we'll comment about why they make sense in light of our observations about the overfitting we've seen thus far.

### Hyperparameter Turning via Grid Search

Next we'll do a grid search over the hyperparameter space we think to be reasonable. Grid searching is a tricky, boring process.

It's _tricky_ because you cannot search the entire hyperparameter space; you have to search a relatively small subspace in order for the grid search to terminate in our lifetimes. Here's my approach to knowing a somewhat reasonable subspace to search:
1. I trained (outside of this notebook) several matrix factorization models one-by-one, examining the results each time. This allowed me to see roughly the reasonable range of some of the hyperparameters. E.g. I realized that a regularization penalty greater than 1e-3 caused the model to cease learning no matter how many latent features I used, so that became a boundary.
2. Knowing the math behind the model will save you some searching time. E.g. I saw that we were dramatically overfitting with a regularization penalty of 1e-8, meaning that trying anything smaller than that would be silly unless we decreased `k`. So to experiment, I kept the regularization parameter at 1e-8, decreased `k` to 2, and saw that we were _still_ overfitting! That sealed the deal: a regularization term less than 1e-8 would never work, so that became a boundary.

Once you find a good hyperparameter subspace to search, you enter the _boring_ part of grid searching. You decide how long you are willing to wait for the grid search to run (in my case, I'm willing to let it run overnight, so 8 hours), then you set it up and let it run. And you _wait_. In my case, from training the models one-by-one above, I know that it takes about 2 minutes to train one model. Back-of-the-envelope estimate: That means I can train ~240 models in 8 hours, which I'll consider an upper bound. I decided to set up the grid in the cell below which will train 150 independent models.

Below we perform the grid search using GraphLab's grid search interface. See the documentation [here](https://turi.com/products/create/docs/generated/graphlab.toolkits.model_parameter_search.grid_search.create.html).

In [6]:
grid_params = {
    'user_id': "user_id",
    'item_id': "joke_id",
    'target': 'rating',
    'num_factors':              [2, 4, 8, 16, 32, 64],            # <-- we're tuning this
    'regularization':           [5e-2, 1e-3, 1e-4, 1e-5, 1e-7],   # <-- ... and this...
    'linear_regularization':    [1e-3, 1e-4, 1e-5, 1e-7, 1e-9],   # <-- ... and this.
    'max_iterations': 100,
    'solver': 'auto',
    'random_seed': 12345,
    'verbose': True
}

def grid_scorer(model, training_set, validation_set):
    factor_predictions = validation_set.to_dataframe().copy()
    factor_predictions['rating'] = model.predict(validation_set)
    s = scoring.score_top_5_percent(factor_predictions)
    r = scoring.score_rmse(factor_predictions)
    return {'score_top_5_percent': s, 'score_rmse': r}

grid = gl.toolkits.model_parameter_search.grid_search.create(
            (gl.SFrame(training_set), gl.SFrame(test_set)),
            gl.factorization_recommender.create,
            grid_params,
            grid_scorer,
            return_model=True,
)

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.job: Creating a LocalAsync environment called 'async'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Jan-26-2017-01-05-1700000' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Jan-26-2017-01-05-1700000' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: A job with name 'Model-Parameter-Search-Jan-26-2017-01-05-1700000' already exists. Renaming the job to 'Model-Parameter-Search-Jan-26-2017-01-05-1700000-c9312'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Jan-26-2017-01-05-1700000-c9312' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Jan-26-2017-01-05-1700000-c9312' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Jan-26-2017-01-05-1700001' ready for execution


### Wait for the grid search to finish

The grid search code above is asynchronous. We use the `grid` object to check the status of the search. See documentation [here](https://turi.com/products/create/docs/generated/graphlab.toolkits.model_parameter_search.ModelSearchJob.html#graphlab.toolkits.model_parameter_search.ModelSearchJob). We can have the notebook block until all models are complete by calling the `get_results()` method.

In [7]:
results = grid.get_results(wait=True)

best_score_top_5_percent = results['score_top_5_percent'].max()

cols_of_interest = ['model_id', 'num_factors', 'regularization', 'linear_regularization',
                    'score_top_5_percent', 'score_rmse']

results[results['score_top_5_percent'] == best_score_top_5_percent][cols_of_interest]

model_id,num_factors,regularization,linear_regularization,score_top_5_percent,score_rmse
47,16,0.001,1e-05,2.76755195131,4.28283319885


If we cared to save the best model in order to deploy it later, here's how we'd do that:

```python
best_model_id = results[results['score_top_5_percent'] == best_score_top_5_percent]['model_id'][0]
best_model = grid.get_models()[best_model_id]
best_model.save('gl_trained_joke_recommender')
```

### Getting the best hyperparameters found by the grid search

Once the grid search is complete, we can query for the best hyperparameters:

Have a look below at the best hyperparameters for maximizing the **top-5-percent score**. Notice the increase in the regularization penalties! That should make sense from our discussion at the beginning about combatting model variance. More regularization -> less variance -> better predictions.

In [8]:
best_params_score_top_5_percent = grid.get_best_params('score_top_5_percent', False)
best_params_score_top_5_percent

{'item_id': 'joke_id',
 'linear_regularization': 1e-05,
 'max_iterations': 100,
 'num_factors': 16,
 'random_seed': 12345,
 'regularization': 0.001,
 'solver': 'auto',
 'target': 'rating',
 'user_id': 'user_id',
 'verbose': True}

Have a look below at the best hyperparamers for minimizing the **test set RMSE**. The model which has the best RMSE score, like the model for top-5-percent, also imposes more regularization penalty, but it only has 2 latent features (`k`)!

In [9]:
best_params_score_rmse = grid.get_best_params('score_rmse', True)
best_params_score_rmse

{'item_id': 'joke_id',
 'linear_regularization': 0.001,
 'max_iterations': 100,
 'num_factors': 2,
 'random_seed': 12345,
 'regularization': 0.0001,
 'solver': 'auto',
 'target': 'rating',
 'user_id': 'user_id',
 'verbose': True}

### Conclusion

We used GraphLab to train many many many matrix factorization recommender models.

We found a model which performs better than the popularity recommender according to the top-5-percent metric, scoring a __2.76755__. This model increased the regularization penalty from GraphLab's default in order to close the overfitting gap a bit.

We also found a _different_ model which minimized the test-set RMSE. This model, like the one above, increased the regularization penatly, but in addition it _decreased_ the number of latent factors (`k`) down to only 2. This closed the overfitting gap even further in order to have the lowest possible test-set RMSE (at least within our search space).